In [1]:
import pandas as pd
import numpy as np

In [8]:
calendar = pd.read_csv('data/calendar.csv')
listing = pd.read_csv('data/listings.csv')

### Add neighbours to calendar

In [11]:
calendar = calendar.rename(columns = {"listing_id":"id"})

In [21]:
calendar = pd.merge(calendar, listing.loc[:,['id','neighbourhood']], on='id')


In [25]:
max(calendar.date)

'2024-03-27'

In [22]:
calendar.to_csv('calendar_with_neighbourhoods.csv')

### make testing set

In [51]:
sampled_calendar = calendar.sample(n=1000)

In [52]:
def clean_and_convert(string):
    # remove commas, dots, and dollar signs
    string = string.replace(',', '').replace('.', '').replace('$', '')

    # convert to float
    number = float(string)

    return number

In [53]:
sampled_calendar['price'] = sampled_calendar.price.apply(clean_and_convert)

In [56]:
grouped_calendar = sampled_calendar.groupby(['neighbourhood','date'])['price'].mean()

In [60]:
grouped_calendar.to_csv('group_calendar.csv')


In [65]:
gp = pd.read_csv('group_calendar.csv')


In [69]:
gp

,neighbourhood,date,price
0,Batignolles-Monceau,2023-04-10,32000.0
1,Batignolles-Monceau,2023-04-11,9950.0
2,Batignolles-Monceau,2023-04-15,8000.0
3,Batignolles-Monceau,2023-04-16,9700.0
4,Batignolles-Monceau,2023-04-19,10000.0
...,...,...,...
925,Élysée,2023-12-18,24000.0
926,Élysée,2024-01-01,142200.0
927,Élysée,2024-02-08,39400.0
928,Élysée,2024-02-26,15000.0


### get avg price per neighbourhood

In [74]:
listing.groupby('neighbourhood')['price'].mean().to_csv('neighbourhood_price.csv')